In [22]:
import argparse
from path import Path

import torch
import torch.backends.cudnn as cudnn
import torch.nn.functional as F
import models
from tqdm import tqdm

import torchvision.transforms as transforms
import flow_transforms
from imageio import imread, imwrite
import numpy as np
from util import flow2rgb


# myimport 
import matplotlib.pyplot as plt


model_names = sorted(name for name in models.__dict__
                     if name.islower() and not name.startswith("__"))


# create the folder for saving the results
path_to_save = Path('my_results')
if not path_to_save.exists():
    path_to_save.mkdir()


# Data loading code
input_transform = transforms.Compose([
    flow_transforms.ArrayToTensor(),
    transforms.Normalize(mean=[0,0,0], std=[255,255,255]),
    transforms.Normalize(mean=[0.411,0.432,0.45], std=[1,1,1])
])

###############importing images and nework form the paths################
path_to_model = "./pretrained_model/flownets_EPE1.951.pth.tar"
path_to_sequesnces = "./sequences-train/"
name = "swan-"

n_img_1 = 1
n_img_2 = 2

path_img1 = path_to_sequesnces + name + str(n_img_1).zfill(3) + ".bmp"
path_img2 = path_to_sequesnces + name + str(n_img_2).zfill(3) + ".bmp"

img1 = imread(path_img1)
img2 = imread(path_img2)

img_pairs = []
img_pairs.append([img1, img2])
print('{} samples found'.format(len(img_pairs)))
###############importing images and nework form the paths################


# setting the cuda device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)


# Importing the model
network_data = torch.load(path_to_model)
print("=> using pre-trained model '{}'".format(network_data['arch']))
model = models.__dict__[network_data['arch']](network_data).to(device)
model.eval()







/tmp/ipykernel_7337/2298950706.py:49: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  img1 = imread(path_img1)
/tmp/ipykernel_7337/2298950706.py:50: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  img2 = imread(path_img2)


1 samples found
cuda
=> using pre-trained model 'flownets'


FlowNetS(
  (conv1): Sequential(
    (0): Conv2d(6, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
    (1): LeakyReLU(negative_slope=0.1, inplace=True)
  )
  (conv2): Sequential(
    (0): Conv2d(64, 128, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
    (1): LeakyReLU(negative_slope=0.1, inplace=True)
  )
  (conv3): Sequential(
    (0): Conv2d(128, 256, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
    (1): LeakyReLU(negative_slope=0.1, inplace=True)
  )
  (conv3_1): Sequential(
    (0): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): LeakyReLU(negative_slope=0.1, inplace=True)
  )
  (conv4): Sequential(
    (0): Conv2d(256, 512, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (1): LeakyReLU(negative_slope=0.1, inplace=True)
  )
  (conv4_1): Sequential(
    (0): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): LeakyReLU(negative_slope=0.1, inplace=True)
  )
  (conv5): Sequential(
    (0): Conv2d(512, 512,